# Fine-movement characterization

In [1]:
""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
import seaborn as sns
from collections import defaultdict
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from datetime import datetime
from scipy.interpolate import interp1d

# --Machine learning and statistics+
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import umap

from one.api import ONE
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'
os.chdir(functions_path)
# from plotting_functions import bins_per_trial_phase, broader_label
from data_processing import process_quiescence
from wheel_functions import wheel_trial_epoch
one = ONE(base_url='https://alyx.internationalbrainlab.org')

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/api.py:1577: UserWarning: Newer cache tables require ONE version 2.7 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


## Load data

In [2]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
os.chdir(data_path)
session_all = pickle.load(open('last_3_training_2024-04-23', 'rb'))

In [5]:
threshold = 0.2 # Need to check if this makes sense
min_period = 400 # This is to match the minimum length of quiescence periods
bin_size = 0.05

# Save data of all sessions for latter
processes_quiescence_all = defaultdict(list)
perf_idx = pd.DataFrame(columns=['performance_easy', 'idx'])

# Loop through animals
mouse_names = list(session_all.keys())
for m, mouse_name in enumerate(mouse_names):
    processes_quiescence_all[mouse_name] = {}

    # Find data of interest
    mouse_data = session_all[mouse_name]
    mouse_session_num = np.max(mouse_data['training_day'])
    use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num - 2]

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        
        session_trials = use_data.loc[use_data['session']==session]
        session_trials['index'] = np.arange(0, len(session_trials))
        
        df = wheel_trial_epoch(one, session_trials, session, bin_size, threshold, min_period).dropna()  
        df = df[['trial', 'wheel', 'movement', 'trial_epoch', 'feedback',
       'next_feedback', 'signed_contrast', 'response', 'reaction', 'choice',
       'probabilityLeft']].drop_duplicates()
    
        processed_quiescence = process_quiescence(df)

        # Save
        processes_quiescence_all[mouse_name][session] = processed_quiescence
        
        # Save performance also
        perf_mouse = pd.DataFrame(columns=['performance_easy', 'idx'], index=range(1))
        perf_mouse['idx'] = session
        perf_mouse['performance_easy'] = np.nanmean(session_trials['correct_easy'])
        
        # Save performance results
        if s == 0:
            perf_idx = perf_mouse
        else:
            perf_idx = perf_idx.append(perf_mouse)

/var/folders/nt/d2j3zp9d1xzb8wgfrw81j0c40000gn/T/ipykernel_86453/2079758226.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session_trials['index'] = np.arange(0, len(session_trials))


KeyError: "None of [Index(['time'], dtype='object')] are in the [columns]"